## Investigating MOT, VES and VCA data

We've got a sample of 1.4 million vehicles that have been downloaded from the VES API - this is an exploration of the data. We will look for:
* how complete the data are
* any anomalies
* anything else that may need doing in the future
* whether the VCA database might be able to help us gain insights on the vehicles' emissions, etc.

In [1]:
# Import packages
import pandas_gbq
from google.oauth2 import service_account
import os
# from google.cloud import bigquery
import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime as dt
import pandas as pd
from tqdm import tqdm

In [2]:
# Credentials for BigQuery
credentials = service_account.Credentials.from_service_account_file('rugged-baton-283921-5706f65c85fe.json')

### Introduction to the data
We have two MOT-derived tables:
* `mots.tests`
* `mots.vehicles`

As the names imply, we have tests, where each row is an MOT test, with details about passes, failures, odometer readings, and we have vehicles, which has some details about vehicles. 

We also have a sample of 1.4 million vehicles details that have been downloaded from the VES API. 

We've performed a join on VES API 1.4 million with `mots.vehicles`, finding the intersection. This has yielded a large table saved in `experimental_tables` called `VES_MOT_vehicles_join`. We used the following SQL query to generate this table:

```
WITH join_condition as (SELECT registration as reg, MAX(registrationDate) as regDate
FROM `rugged-baton-283921.mots.vehicles` 
WHERE registration IN (SELECT registrationNumber FROM `rugged-baton-283921.mots.VES_results`)
GROUP BY registration
),
mots_vehicles as (
    SELECT *
    FROM `rugged-baton-283921.mots.vehicles` A
    JOIN join_condition B
    ON A.registration = B.reg AND A.registrationDate = B.regDate 
) SELECT *
FROM mots_vehicles A
JOIN `rugged-baton-283921.mots.VES_results` B
ON A.registration = B.registrationNumber;``
```

This query takes into account the problem of reused number plates. In short, it:
* creates a table with two columns,  a VRN and the latest registration date available in the database (see note below)
* it finds the intersection between the two-column table created above with the `mots.vehicles` table, so that it extracts only the youngest vehicles using any particular VRN in the case of a reused plate,
* lastly, it joins the results of the intersection above with the details from the VES API using VRN as the key. 

Note: we take the latest registration date because the VES API only contains the details of the latest vehicle to use a particular VRN, and indeed it only has vehicles in the database that are current (i.e. they have not been scrapped or written off). 

The result of the above SQL query gives us a table with the following schema:


Field name | Type |
---|---|
registration|STRING|
make|STRING|
model|STRING|
firstUsedDate|TIMESTAMP|
registrationDate|TIMESTAMP|
manufactureDate|TIMESTAMP|
engineSize|FLOAT|
fuelType|STRING|
primaryColour|STRING|
vehicleId|STRING|
jsonPage|STRING|
reg|STRING|
regDate|TIMESTAMP|
registrationNumber|STRING|
co2Emissions|STRING|
engineCapacity|STRING|
markedForExport|STRING|
fuelType_1|STRING|
motStatus|STRING|
colour|STRING|
make_1|STRING|
typeApproval|STRING|
yearOfManufacture|STRING|
taxDueDate|STRING|
taxStatus|STRING|
dateOfLastV5CIssued|STRING|
motExpiryDate|STRING|
wheelplan|STRING|
monthOfFirstRegistration|STRING|
revenueWeight|STRING|
monthOfFirstDvlaRegistration|STRING|
euroStatus|STRING|
realDrivingEmissions|STRING|
artEndDate|STRING|

### Pull the data into pandas

In [3]:
# We're going to leave out one or two fields, but we'll take almost everything.
sql = """
SELECT registration, make, model, registrationDate, manufactureDate, engineSize, fuelType, primaryColour, vehicleID, regDate, co2Emissions, engineCapacity, fuelType_1, motStatus, colour, make_1, typeApproval, yearOfManufacture, taxStatus, dateOfLastV5CIssued, motExpiryDate, wheelplan, monthOfFirstDvlaRegistration, monthOfFirstRegistration, revenueWeight, euroStatus, realDrivingEmissions
FROM `rugged-baton-283921.experimental_tables.VES_MOT_vehicles_join`;
"""

In [4]:
df = pandas_gbq.read_gbq(sql, project_id='rugged-baton-283921', credentials = credentials, progress_bar_type='tqdm')

Downloading: 100%|███████████████████████████████████████████████████████| 1442478/1442478 [04:18<00:00, 5589.91rows/s]


A first look at the suggests all is well. "

In [6]:
df.head()

,registration,make,model,registrationDate,manufactureDate,engineSize,fuelType,primaryColour,vehicleID,regDate,...,yearOfManufacture,taxStatus,dateOfLastV5CIssued,motExpiryDate,wheelplan,monthOfFirstDvlaRegistration,monthOfFirstRegistration,revenueWeight,euroStatus,realDrivingEmissions
0,LR52UEK,MERCEDES,C 220,2002-09-06 00:00:00+00:00,2002-09-06 00:00:00+00:00,2148.0,Diesel,Silver,Br0QoyRvaRBFR0ytUDPOaQ==,2002-09-06 00:00:00+00:00,...,2002,Untaxed,2018-05-22,2020-09-16,2 AXLE RIGID BODY,None,2002-09,None,None,None
1,AJ07EYY,MITSUBISHI,LANCER,2007-06-01 00:00:00+00:00,2007-06-01 00:00:00+00:00,1584.0,Petrol,Blue,L1MlYA8w2N8e8jAaSsfgsQ==,2007-06-01 00:00:00+00:00,...,2007,Taxed,2019-07-31,2021-09-03,2 AXLE RIGID BODY,None,2007-06,None,None,None
2,DE51XBP,VAUXHALL,CORSA,2001-10-31 00:00:00+00:00,2001-10-31 00:00:00+00:00,973.0,Petrol,Blue,-pworJ79zmc5AIrJ598_Ww==,2001-10-31 00:00:00+00:00,...,2001,SORN,2020-02-03,2021-05-20,2 AXLE RIGID BODY,None,2001-10,None,None,None
3,V116HWA,VOLKSWAGEN,TRANSPORTER,2000-02-23 00:00:00+00:00,2000-02-23 00:00:00+00:00,2370.0,Diesel,Blue,hgUxBRE_kMcC05wpojLkeg==,2000-02-23 00:00:00+00:00,...,2000,Taxed,2019-09-25,2021-03-28,2 AXLE RIGID BODY,2014-04,2000-02,None,None,None
4,WG61KWS,NISSAN,NAVARA,2012-02-01 00:00:00+00:00,2012-02-01 00:00:00+00:00,2488.0,Diesel,Black,TVt_irkIv8wN1GMk_PjNXA==,2012-02-01 00:00:00+00:00,...,2012,Taxed,2017-06-24,2021-07-22,2 AXLE RIGID BODY,None,2012-02,3210,None,None


Data types have been preserved from BigQuery:

In [11]:
df.dtypes

registration                                 object
make                                         object
model                                        object
registrationDate                datetime64[ns, UTC]
manufactureDate                 datetime64[ns, UTC]
engineSize                                  float64
fuelType                                     object
primaryColour                                object
vehicleID                                    object
regDate                         datetime64[ns, UTC]
co2Emissions                                 object
engineCapacity                               object
fuelType_1                                   object
motStatus                                    object
colour                                       object
make_1                                       object
typeApproval                                 object
yearOfManufacture                            object
taxStatus                                    object
dateOfLastV5

In [12]:
df.isna().sum()

registration                          0
make                                  0
model                              3646
registrationDate                      0
manufactureDate                     218
engineSize                         2286
fuelType                              0
primaryColour                         0
vehicleID                             0
regDate                               0
co2Emissions                      86026
engineCapacity                      182
fuelType_1                           21
motStatus                             0
colour                                0
make_1                                0
typeApproval                     111849
yearOfManufacture                   226
taxStatus                             0
dateOfLastV5CIssued                   1
motExpiryDate                      1717
wheelplan                             0
monthOfFirstDvlaRegistration    1429596
monthOfFirstRegistration              0
revenueWeight                    938442
